# Table of Contents
 <p><div class="lev1 toc-item"><a href="#Initialize-Environment" data-toc-modified-id="Initialize-Environment-1"><span class="toc-item-num">1&nbsp;&nbsp;</span>Initialize Environment</a></div><div class="lev2 toc-item"><a href="#Generate-List-of-Data" data-toc-modified-id="Generate-List-of-Data-11"><span class="toc-item-num">1.1&nbsp;&nbsp;</span>Generate List of Data</a></div><div class="lev1 toc-item"><a href="#Subgraph-Topography" data-toc-modified-id="Subgraph-Topography-2"><span class="toc-item-num">2&nbsp;&nbsp;</span>Subgraph Topography</a></div><div class="lev2 toc-item"><a href="#Relate-Subgraph-Connection-Strength-to-Connection-Distance" data-toc-modified-id="Relate-Subgraph-Connection-Strength-to-Connection-Distance-21"><span class="toc-item-num">2.1&nbsp;&nbsp;</span>Relate Subgraph Connection Strength to Connection Distance</a></div><div class="lev2 toc-item"><a href="#Plot-Ranked-Topographical-Sensitivity" data-toc-modified-id="Plot-Ranked-Topographical-Sensitivity-22"><span class="toc-item-num">2.2&nbsp;&nbsp;</span>Plot Ranked Topographical Sensitivity</a></div>

# Initialize Environment

In [ ]:
try:
    %load_ext autoreload
    %autoreload 2
    %reset
except:
    print 'NOT IPYTHON'

from __future__ import division

import os
import sys
import glob

import numpy as np
import pandas as pd
import seaborn as sns
import scipy.stats as stats
import scipy.io as io
import h5py
import matplotlib.pyplot as plt
from matplotlib import rcParams

sys.path.append('/Users/akhambhati/Developer/hoth_research/Echobase')
import Echobase
convert_conn_vec_to_adj_matr = Echobase.Network.Transforms.configuration.convert_conn_vec_to_adj_matr
convert_adj_matr_to_cfg_matr = Echobase.Network.Transforms.configuration.convert_adj_matr_to_cfg_matr

rcParams = Echobase.Plotting.fig_format.update_rcparams(rcParams)

path_AtlasData = '/Users/akhambhati/Remotes/CORE.MRI_Atlases'
path_CoreData = '/Users/akhambhati/Remotes/CORE.fMRI_multiband.mmattar/restdata'
path_PeriphData = '/Users/akhambhati/Remotes/RSRCH.NMF_Subnetworks'
path_SubData = path_PeriphData + '/e03d-DynFuncSubgraph-Population'
path_ExpData = path_PeriphData + '/e05d-CmpTopographySubgraph-Population'

for path in [path_CoreData, path_PeriphData, path_SubData, path_ExpData]:
    if not os.path.exists(path):
        print('Path: {}, does not exist'.format(path))
        os.makedirs(path)     
        
if not os.path.exists('./e05d-Figures'):
    os.makedirs('./e05d-Figures')

## Generate List of Data

In [ ]:
data_dict = {'subgraph_path': '{}/NMF_Optimization.consensus_subgraph.npz'.format(path_SubData),
             'cfg_path': '{}/NMF_Optimization.CfgMatr.npz'.format(path_SubData),
             'subgraph_geom_null_path': glob.glob('{}/NMF_GeomNull.subgraph_seed-*.npz'.format(path_SubData))}

dist_matr = np.load('{}/geom_adj_matr.npz'.format(path_SubData))['dist_matr']  
dist_conn = convert_adj_matr_to_cfg_matr(np.expand_dims(dist_matr, axis=0)).reshape(-1)
roi_csv = pd.read_csv('{}/Atlas/HOA112_Labels.csv'.format(path_CoreData))

# Subgraph Topography

## Relate Subgraph Connection Strength to Connection Distance

In [ ]:
# Get intra-hemispheric connections
def get_intra_hemi(matr):
    intra_Rhemi_ix, intra_Rhemi_iy = np.meshgrid(np.flatnonzero(roi_csv['Hemisphere'] == 'R'),
                                               np.flatnonzero(roi_csv['Hemisphere'] == 'R')) 
    
    intra_Lhemi_ix, intra_Lhemi_iy = np.meshgrid(np.flatnonzero(roi_csv['Hemisphere'] == 'L'),
                                               np.flatnonzero(roi_csv['Hemisphere'] == 'L')) 
    
    inter_hemi_ix, inter_hemi_iy = np.meshgrid(np.flatnonzero(roi_csv['Hemisphere'] == 'L'),
                                               np.flatnonzero(roi_csv['Hemisphere'] == 'R')) 

    conn_Rhemi = convert_adj_matr_to_cfg_matr(np.expand_dims(matr[intra_Rhemi_ix, intra_Rhemi_iy], axis=0)).reshape(-1)
    conn_Lhemi = convert_adj_matr_to_cfg_matr(np.expand_dims(matr[intra_Lhemi_ix, intra_Lhemi_iy], axis=0)).reshape(-1)
    conn_hemi = convert_adj_matr_to_cfg_matr(np.expand_dims(matr[inter_hemi_ix, inter_hemi_iy], axis=0)).reshape(-1)
    
    conn_intra = np.concatenate((conn_Rhemi, conn_Lhemi))
    conn_inter = np.concatenate((conn_hemi, conn_hemi))
    
    return conn_intra, conn_inter


%matplotlib inline

# Get Data
data_sub = np.load(data_dict['subgraph_path'], mmap_mode='r')
fac_subgraph = data_sub['fac_subnet']
fac_coef = data_sub['fac_coef']

# Get Parameters
n_fac = fac_subgraph.shape[0]
n_conn = fac_subgraph.shape[1]
n_null = len(data_dict['subgraph_geom_null_path'])
dist_intra, dist_inter = get_intra_hemi(dist_matr)

# Setup buckets
tsi_intra_real = np.nan*np.zeros((n_fac))
tsi_intra_geom_null = np.nan*np.zeros((n_fac, n_null))

tsi_inter_real = np.nan*np.zeros((n_fac))
tsi_inter_geom_null = np.nan*np.zeros((n_fac, n_null))


# Correlation between Subgraph and Distance Matrix
subg_dist_intra = []
subg_dist_inter = []
for f_id in xrange(n_fac):
    subg_intra, subg_inter = get_intra_hemi(convert_conn_vec_to_adj_matr(fac_subgraph[f_id, :]))
    subg_dist_intra.append(subg_intra)
    subg_dist_inter.append(subg_inter)    
    
    rho, pval = stats.pearsonr(dist_intra, subg_intra)
    tsi_intra_real[f_id] = rho
    rho, pval = stats.pearsonr(dist_inter, subg_inter)
    tsi_inter_real[f_id] = rho

# Geometric Null Correlation between Subgraph and Distance Matrix
for n_i, null_path in enumerate(data_dict['subgraph_geom_null_path']):
    data_null_sub = np.load(null_path, mmap_mode='r')
    fac_subgraph = data_null_sub['fac_subnet']

    for f_id in xrange(n_fac):
        subg_intra, subg_inter = get_intra_hemi(convert_conn_vec_to_adj_matr(fac_subgraph[f_id, :]))

        rho, pval = stats.pearsonr(dist_intra, subg_intra)
        tsi_intra_geom_null[f_id, n_i] = rho
        rho, pval = stats.pearsonr(dist_inter, subg_inter)
        tsi_inter_geom_null[f_id, n_i] = rho

np.savez('{}/Topographic_Sensitivity.npz'.format(path_ExpData),
         tsi_intra_real=tsi_intra_real,
         tsi_intra_geom_null=tsi_intra_geom_null,
         tsi_inter_real=tsi_inter_real,
         tsi_inter_geom_null=tsi_inter_geom_null)

## Plot Ranked Topographical Sensitivity

In [ ]:
# Load Data
df = np.load('{}/Topographic_Sensitivity.npz'.format(path_ExpData))

tsi_intra_real = df['tsi_intra_real']
tsi_intra_geom_null = df['tsi_intra_geom_null']

tsi_inter_real = df['tsi_inter_real']
tsi_inter_geom_null = df['tsi_inter_geom_null']

n_fac, n_null = tsi_intra_geom_null.shape

### Plot Intra-Hemisepheric Distribution
for hemi_name, tsi_real, tsi_geom_null in [('Intra_Hemi', tsi_intra_real, tsi_intra_geom_null),
                                           ('Inter_Hemi', tsi_inter_real, tsi_inter_geom_null)]:
    plt.figure(figsize=(3,3))
    ax = plt.subplot(111)

    # Real Topography
    ax.plot(np.arange(n_fac)+1, np.sort(tsi_real),
            color=[0.2, 0.2, 0.2])

    flat_tsi_geom_null = tsi_geom_null.reshape(-1)
    alpha = 0.025
    for ix, tsi in enumerate(np.sort(tsi_real)):
        if np.nanmean(flat_tsi_geom_null > tsi) < alpha:
            color=[0.20, 0.20, 0.75] 
        elif np.nanmean(flat_tsi_geom_null < tsi) < alpha:
            color=[0.75, 0.20, 0.20]
        else:
            color=[0.20, 0.20, 0.20]

        ax.scatter(ix+1, tsi,
                   lw=0, color=color, alpha=1.0, s=40.0)

    # Geometric Null Topography
    ax.fill_between(np.arange(1, n_fac+1),
                    y1=np.percentile(flat_tsi_geom_null[~np.isnan(flat_tsi_geom_null)], alpha*100),
                    y2=np.percentile(flat_tsi_geom_null[~np.isnan(flat_tsi_geom_null)], (1-alpha)*100),
                    color=[1.00, 0.25, 0.25], alpha=0.2, lw=0)

    # Axis Settings
    ax.set_xlim([0, n_fac+1])
    ax.set_ylim([-0.5, 0.5])
    ax.yaxis.set_ticks_position('left')
    ax.xaxis.set_ticks_position('bottom')
    ax.set_ylabel('Pearson Correlation\n(Edge Strength vs Euclidean Dist.)');
    ax.set_xlabel('Subgraphs');
    ax.set_title(hemi_name)

    plt.savefig('./e05d-Figures/Topographic_Sensitivity-{}.svg'.format(hemi_name))
    plt.show()
    plt.close()              

In [ ]:
from Echobase.Plotting import render_brain_connectivity

# Load Subgraph Data
data_sub = np.load(data_dict['subgraph_path'], mmap_mode='r')
fac_subgraph = data_sub['fac_subnet']

df = np.load('{}/Topographic_Sensitivity.npz'.format(path_ExpData))
tsi_intra_real = df['tsi_intra_real']
tsi_inter_real = df['tsi_inter_real']

sel_fac = [('IntraH_Short', np.argsort(tsi_intra_real)[1]),
           ('IntraH_Long', np.argsort(tsi_intra_real)[-2]),
           ('InterH_Short', np.argsort(tsi_inter_real)[1]),
           ('InterH_Long', np.argsort(tsi_inter_real)[-2])] 

# Read the ROI CSV
roi_csv = pd.read_csv('{}/fsl/HarvardOxford/HOA112_Labels.csv'.format(path_AtlasData))
roi_coords = np.vstack((roi_csv['X'], roi_csv['Y'], roi_csv['Z'])).T
node_rgba = [(0.3, 0.69, 1.0, 1.0) for rr in roi_coords]

view_angle = {'Axial_LR': [0.0, 0.0],
              'Axial_RL': [0.0, 180.0],
              'Sag_PA': [0.0, 90.0],
              'Sag_AP': [180.0, 90.0]}

for subg_type, ix in sel_fac:
    render_brain_connectivity.draw(vtk_files=['{}/fsl/MNI/rh.pial.vtk'.format(path_AtlasData),
                                              '{}/fsl/MNI/lh.pial.vtk'.format(path_AtlasData)],
                                   brain_rgba=(0.5, 0.5, 0.5, 0.05), conn_cmap='plasma',
                                   node_coords=roi_coords, node_rgba=node_rgba, conn_thr=[0.975, 1.0],
                                   conn_list=fac_subgraph[ix, :] / fac_subgraph[ix, :].max())
    for ang in view_angle.keys():
        render_brain_connectivity.mlab.view(azimuth=view_angle[ang][0],
                                                              elevation=view_angle[ang][1])
        render_brain_connectivity.mlab.savefig('./e05d-Figures/Subgraph-{}-{}.png'.format(subg_type, ang))
    render_brain_connectivity.mlab.close(all=True)